# Local TIG Algorithm Tester

Tests algorithms available for benchmarking in the current round on local compute, to determine the most optimal algorithm for benchmarking
Settings:
- Length of time testing each algorithm (Default 30s)
- Difficulty scaling (Default minimum scaled frontier)
- Number of cores used to test (Default 1 core)

In [1]:
#importing modules

import ipywidgets as widgets
import requests
import urllib
import re
import multiprocessing
from time import sleep
from itertools import repeat
from tools import loop_entry, get_results, build_worker

## Pulling Round Data
If an error shows, run again as sometimes the server is not as cooperative until "Complete" is printed

In [2]:
# pulls data from the current block and round
url = "https://mainnet-api.tig.foundation/"
block_data = requests.get(f"{url}get-block").json()
latest_block = block_data["block"]["id"]

challenge_data = requests.get(f"{url}get-challenges?block_id={latest_block}").json()
algorithm_data = requests.get(f"{url}get-algorithms?block_id={latest_block}").json()

challenge_id_dict = {challenge["id"]:[challenge["details"]["name"]] for challenge in challenge_data["challenges"]}
order_dict = {challenge["id"]:i for (i, challenge) in enumerate(challenge_data["challenges"])}
algorithm_id_dict = {algorithm["id"]:algorithm["details"]["name"] for algorithm in algorithm_data["algorithms"] if algorithm["state"]["round_pushed"]}
[challenge_id_dict[algorithm["id"][:4]].append(algorithm["id"]) for algorithm in algorithm_data["algorithms"] if algorithm["state"]["round_pushed"]]

print("Complete")

Complete


## Testing Algorithms

Tests every algorithm for each challenge, and returns the algorithm with the lowest time per solution for each challenge

In [3]:
#Select time in seconds to test each algorithm within range (10, 900)

testing_time = widgets.interactive(lambda x=30:x, x=(10,900,10))
display(testing_time)

interactive(children=(IntSlider(value=30, description='x', max=900, min=10, step=10), Output()), _dom_classes=…

In [4]:
#Select difficulty to be minimum in current scaled frontier or suggested algorithm testing difficulties (much lower difficulty, expecting more results)
#WARNING lower difficulty may benefit "greedy" algorithms that may not perform as well at higher difficulties
unscaled_difficulties = {"satisfiability":[50, 300], "vehicle_routing":[40, 250], "knapsack":[50, 10]}

difficulty_selector = widgets.interactive(lambda x:x, x=[("Scaled difficulty", "scaled"), ("Lower difficulty", "unscaled")])
display(difficulty_selector)

interactive(children=(Dropdown(description='x', options=(('Scaled difficulty', 'scaled'), ('Lower difficulty',…

In [5]:
#Select the number of cores you want to use during the tests (out of your maximum cores)
#Cores are used to batch test algorithms, reducing the overall time and allowing longer timers son each algorithm test in the same overall time
num_cores = widgets.interactive(lambda x=1:x, x=(1, multiprocessing.cpu_count(),1))
display(num_cores)

interactive(children=(IntSlider(value=1, description='x', max=16, min=1), Output()), _dom_classes=('widget-int…

In [6]:
#Run this cell until "Complete" is printed out
%env RUSTFLAGS=-Awarnings

with open("test_results.txt", "w") as file:
    file.write("New test\n")

build_worker()

for challenge in challenge_id_dict.keys():
    
    #Set challenge environment
    %env CHALLENGE={challenge_id_dict[challenge][0]}

    #Set difficulty to minimum in scaled frontiers and dumping into json file
    if difficulty_selector.result == "scaled":
        difficulty = sorted(challenge_data["challenges"][order_dict[challenge]]["block_data"]["scaled_frontier"])[0]
    else:
        difficulty = unscaled_difficulties[challenge_id_dict[challenge][0]]
    with open("settings.json", "w+") as file:
            settings = '{' + f'"block_id": "","algorithm_id": "","challenge_id": "","player_id": "","difficulty": {difficulty}' + '}'
            file.write(settings)

    
    #looping through all available algorithms for benchmarking in
    for algorithm in challenge_id_dict[challenge][1:]:

        #Downloading wasm blob
        branch_name = f"{challenge_id_dict[challenge][0]}/{algorithm_id_dict[algorithm]}"
        url = f"https://raw.githubusercontent.com/tig-foundation/tig-monorepo/{branch_name}/tig-algorithms/wasm/{branch_name}.wasm"
        response = requests.get(url)
        with open(f"wasm/{algorithm_id_dict[algorithm]}.wasm", "wb") as file:
            file.write(response.content)
        print(f"Downloaded {branch_name}")

        raw_result = [0,0,0]

        test_result = {"name":algorithm_id_dict[algorithm]}

        with multiprocessing.Manager() as manager:
            raw_result = manager.list([0,0,0])
            workers = [multiprocessing.Process(target=loop_entry, args=[algorithm_id_dict[algorithm], raw_result, testing_time.result, ]) for _ in range(num_cores.result)]

            [worker.start() for worker in workers]
            [worker.join() for worker in workers]
            [worker.close() for worker in workers]
            
            print(raw_result)
            test_result = get_results(test_result=test_result, raw_result=raw_result)
            if test_result["solutions"] != 0:
                test_result["avg_time_per_solution"] = (testing_time.result / test_result["solutions"]) * 1000
            else:
                test_result["avg_time_per_solution"] = 0

        with open("test_results.txt", "a") as file:
            file.write(f"Algorithm: {test_result["name"]}, Instances: {test_result["instances"]}, solutions: {test_result["solutions"]}, invalid_solutions: {test_result["invalid_solutions"]}, errors: {test_result["errors"]}, avg_time_per_solution: {test_result["avg_time_per_solution"]}ms \n")

print("Complete")

env: RUSTFLAGS=-Awarnings
tig-worker successfully built
env: CHALLENGE=vehicle_routing
Downloaded vehicle_routing/lazier_cw
[8419, 239, 0]
Downloaded vehicle_routing/simulated_annealing
[374, 0, 0]
Downloaded vehicle_routing/genetic_algorithm
[8996, 0, 10534]
Downloaded vehicle_routing/overloded
[34, 0, 0]
Downloaded vehicle_routing/filter_cw
[10187, 13, 0]
Downloaded vehicle_routing/clarke_wright
[9213, 235, 0]
Downloaded vehicle_routing/aco_sbas
[30, 0, 30]
Downloaded vehicle_routing/fast_exit_clarke


Process Process-79:
Process Process-80:
Process Process-81:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-88:
Process Process-82:
Process Process-86:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-83:
Traceback (most recent call last):
Process Process-84:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-85:
  File "/Users/jack/.pyenv/versions/3.12.0/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/jack/.pyenv/versions/3.12.0/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jack/TIG/TIG-algorithm-tester/local-tester/tools.py", line 15, in loop_entry
    raw_result[0] += 1
    ~~~~~~~~~~^^^
  File "<string>", line 2, in __getitem__
  File "/Users/jack/.pyenv/versions/3.12.0/lib/python3.12/multiprocessing/managers.py", l

KeyboardInterrupt: 

In [ ]:
#Filtering compiler messages and printing out results
with open("test_results.txt", "r") as file:
    captured = file.readlines()

test_data = [x for x in captured if "solution" in x]

for challenge in challenge_id_dict.keys():

    tested_algorithms = test_data[:(len(challenge_id_dict[challenge])-1)]
    test_data = test_data[(len(challenge_id_dict[challenge])-1):]
    
    results_dict = {float(re.search(r"(?:avg_time_per_solution: )(\d*\.?\d+)(?:ms)", test).groups()[0]):re.search(r"(?:Algorithm: )(\w+)(,)", test).groups()[0] for test in tested_algorithms}
    
    try:
        fastest_speed = min([x for x in results_dict.keys() if x != 0])
    except ValueError:
        print(f"For {challenge_id_dict[challenge][0]}, no solutions were found using any algorithnms - try a lower difficulty or a longer testing timer")
    else:
        best_algo = results_dict[fastest_speed]
    
        if difficulty_selector.result == "scaled":
            difficulty = sorted(challenge_data["challenges"][order_dict[challenge]]["block_data"]["scaled_frontier"])[0]
        else:
            difficulty = unscaled_difficulties[challenge_id_dict[challenge][0]]
    
        print(f"Best algorithm for {challenge_id_dict[challenge][0]} is {best_algo}, with an avg_time_per_solution of {fastest_speed}ms, tested with difficulty: {difficulty}")
